In [598]:
import pandas as pd
import json
from pymongo import MongoClient
from pandas.io.json import json_normalize
import pandas as pd
from IPython.display import Image

        
#import the data 
Starbucks=pd.read_json("INPUT/Starbucks1.json")
Veg=pd.read_json("INPUT/Veg1.json")
Comp=pd.read_json("INPUT/comp1.json")
Design=pd.read_json("INPUT/design1.json")
Airports=pd.read_json("INPUT/Airports1.json")
CompRecent=pd.read_json("INPUT/Comp11.json")

#this database includes All the different category_codes documents (all together for the map plots)
BigComp=pd.read_json("INPUT/alltot_processed.json")
events=pd.read_json("INPUT/Events.json")


# First step: finding the city

In the file ProjectW4 I run a deep analysis of the crunchbase dataset using MongoDB.
The first goal of the project was:
    "Your goal is to place the **new company offices** in the best place for the company to grow".
I chose Los Angeles.

The highest number of tech start ups are settled in California (see picture below), in the San Francisco area.
Filtering the crunchbase for recent companies (founded year between 2010-2018) with similar specifics to my company : 
    {"founded_year":{"$gte":2010}},
    {"founded_year":{"$lte":2018}},
    {"category_code": searchre},
    {"deadpooled_year": None},
    {"number_of_employees":{"$gte":60}},
    {"number_of_employees":{"$lte":100}}
I could see that most of my successful competitors in Game_video were settled around Los Angeles-Beverly hills area like Play station,Social Gaming... 
https://www.builtinla.com/2016/10/24/video-game-studios-know
https://www.builtinla.com/companies/type/gaming-companies-los%20angeles


<img style="float: left;" src="OUTPUT/nume.png">

From the graph below can be seen that the avarage total money raised [USD] is not very high in California,
in fact most of the startups are very small and there are a few outliers.
But if we restrict to the Gaming industry the avarage for small company is higher.
Claifornia anyway is in the top five States as for avarage total money raised.


<img style="float: left;" src="OUTPUT/box.png">

<img style="float: left;" src="OUTPUT/money.png">


<img style="float: left;" src="OUTPUT/recent.png ">

After having found the city that with the best odds about monetary success (taking into account my current dimensions),the next step is to find the optimal location within the city that would benefit the most my employees.

I've used some API to fetch informations about the Starbucks, Airports, Recent Companies, Vegan restaurants, 
parties and events in the city and I build some dataframes with the queries results always insluding geospatial coordinates.

Taking advantage of the fact that MongoDB allows me to create Collections with different documents' categories inside, I created a dataset including all the different categories called "geopro". This will help me in my approach 2 solution.

In order to find the location I tried 2 approaches.

# Approach 1: computation of the baricenter of the coordinates for the 5 df

In this approach I calculated the 5 baricenter for the different categories categories(Starbucks, Vegan restaurant, Airport, Design company, Recent company ). 
Then I calculated the target coordinates as the weighted distance from these 5 baricentres.
The weights are computed according to the number of employees having benefits from being close to one category istance.
- 20 Designers           -->W(Design)= 20/133
- 5 UI/UX Engineers      -->W(UI/UX Engineers)= 20/133
- 10 Frontend Developers -->W(Frontend Developers)= 20/133
- 15 Data Engineers      -->W(Data Engineers)= 20/133
- 5 Backend Developers   -->W(Backend Developers)= 20/133
- 20 Account Managers    -->W(Account Managers)= 20/133
- 1 Maintenance guy.     -->W(Maintenance guy)= 1/133
- 10 Executives          -->W(Executives)= 10/133
- 1 CEO/President        -->W(CEO)= 1/133

## computation of the 5 baricentres

In [135]:
StarbucksBaricenter=[Starbucks["latitude"].mean(),Starbucks["longitude"].mean()]#10 executives
VegBaricenter=[Veg["latitude"].mean(),Veg["longitude"].mean()] #1 CEO
CompBaricenter=[CompRecent["latitude"].mean(),CompRecent["longitude"].mean()]#82 all the employees
DesignBaricenter=[Design["latitude"].mean(),Design["longitude"].mean()]# 20 Designers
AirportsBaricenter=[Airports["latitude"].mean(),Airports["longitude"].mean()]#20 Account Managers

## computation of the weighted distance

In [136]:

coordinates=[0,0]
coordinates[0]=(StarbucksBaricenter[0]*10/133+VegBaricenter[0]*1/133+CompBaricenter[0]*82/133+DesignBaricenter[0]*20/133+AirportsBaricenter[0]*20/133)
coordinates[1]=(StarbucksBaricenter[1]*10/133+VegBaricenter[1]*1/133+CompBaricenter[1]*82/133+DesignBaricenter[1]*20/133+AirportsBaricenter[1]*20/133)
coordinates


[34.04417483571285, -118.33669586043084]

In [505]:
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML, display


lat=34.04417483571285
long=-118.33669586043084
create_map(lat,long,dbTot)


This solution might not be optimal, so I tried a second approach.

# Approach 2: using mongoDB

I tested 100000 random locations within Los Angeles borders, and calculated a score for each point.
The score is the weighted sum of the distances between my random point and the closest element of the different categories(Starbucks, Vegan restaurant, Airport, Design company, Recent company ). 

In [373]:
dbName = "geopro"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()
cur = db.list_collections()

In [374]:
#creation of a 2dsphere index
from pymongo import MongoClient, GEOSPHERE
db.geoprog.create_index( [ ("office" , GEOSPHERE)]  )


'office_2dsphere'

In [557]:
import random
#100 random points within Los Angeles border line
lats= [33.99 + random.random()*( 34.1129-33.99) for n in range(10000)]
longs= [-118.17 - random.random()*( 118.45 -118.17) for n in range(10000)]

categories=["airport","Starbucks","VeganRest","design","company"]
weights={"airport":133/20,"Starbucks":133/10,"VeganRest":133,"design":133/20,"company":133/82}#the smallest the score the best


In [558]:
res=bestCoord(lats,longs)
print(res)

#[ -118.353018118,34.100]34.03222952754255, -118.29954303751526 34.14070448441408, -118.20454896353614

[34.063371824427016, -118.2842830534464]


## display of the result 


In [559]:
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML, display

#add of a colors column to the overall database
create_map(res[0],res[1],dbTot)



The point found with the approach 2 is very close to the point found with approach 1. So I think it is a good location for my company.

In [ ]:
#display of the closest places for each category_code to my company

In [571]:
point={"type":"Point","coordinates":[  res[1],res[0]]}
nearest=nearestpleces(point)
dfNear=pd.DataFrame(nearest)
dfNear

,latitude,longitude,name,category_code
0,33.922840,-118.335187,Jack Northrop Field/Hawthorne Municipal Airport,airport
1,34.061490,-118.306030,Starbucks,Starbucks
2,34.080810,-118.270390,Vegan House,VeganRest
3,34.101730,-118.344390,Kobe Digital,design
4,34.051409,-118.254558,Stickam,company


In [572]:
create_map(res[0],res[1],dfNear)

The map shows the closest places to my company coordinates for the different category_codes.
This approach, after several tentatives, provides a better solution.

In [593]:
import requests
import json
import pprint 

coordinates= str(res[0])+"," +str(res[1])
  
resp = requests.get(f"https://geocode.xyz/{coordinates}",params={"json":1})
json = resp.json()
pprint.pprint(json)


{'alt': {'loc': [{'city': 'Los Angeles',
                  'class': {},
                  'latt': '34.06357',
                  'longt': '-118.28455',
                  'postal': '90020',
                  'prov': 'CA',
                  'staddress': 'West 6th Street',
                  'stnumber': '2900'},
                 {'city': 'Los Angeles',
                  'class': {},
                  'dist': '0.033',
                  'latt': '34.06357',
                  'longt': '-118.28455',
                  'postal': '90020',
                  'prov': 'CA',
                  'staddress': 'West 6th Street',
                  'stnumber': '2900'},
                 {'city': 'Los Angeles',
                  'class': {},
                  'dist': '0.055',
                  'latt': '34.0633',
                  'longt': '-118.28369',
                  'postal': '90057',
                  'prov': 'CA',
                  'staddress': 'West 6th Street',
                  'stnumber': '2820'},
    

This is a list of the events close to the company location coordinates with the distance (km)

In [600]:
events[["name",'distance']]


,name,distance
0,Snow Patrol Acoustic Tour,0.97
1,Snow Patrol Acoustic Tour,0.97
2,Netflix Is A Joke Fest Presents: Taylor Tomlinson,0.45
3,Passenger,0.97
4,Netflix Is A Joke Fest Presents: Catherine Cohen,0.45
5,Netflix Is A Joke Fest Presents: London Hughes,0.45
6,Netflix Is A Joke Fest Presents: Cristela Alonzo,0.45
7,Netflix Is A Joke Fest Presents: Endless Honey...,0.45
8,Netflix Is A Joke Fest Presents: Mo Gilligan,0.45


# functions used in the report

In [ ]:
from math import sin, cos, sqrt, atan2, radians

#point={"type":"Point","coordinates":[ -118.33669586043084,34.04417483571285]}


def findfirst(point,category):
     return db.geoprog.find_one( {"$and":
                              [{ "office" :
                                 { "$near" :
                                   { "$geometry" : point,
                                         "$maxDistance" : 2000} } },
                               {"category_code":f"{category}"}]} )

def distance(lat1, lon1, lat2, lon2):  
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [ ]:
distanza={}
def score(point):
    somma=0
    for cat in categories:
        try:
            el=findfirst(point,cat)
            lat=el["latitude"]#el["office"]["coordinates"][1]
            lon=el["longitude"]#el["office"]["coordinates"][0]
            distanza[cat]=(5000-distance(lat,lon,point["coordinates"][1],point["coordinates"][0]))*weights[cat]*0.01
            somma+=distanza[cat]
        except:
            continue
            somma+=10000000000
    return somma

#the best point is the one that minimize the score


def bestCoord(lats,longs):  
    count=0
    scores=dict()
    point=dict()
    for i in range(len(lats)):      
        point["type"]="Point"
        point["coordinates"]=[longs[i],lats[i]]
        scores[count]=[score(point)]
        count+=1
    bestscore=min(scores.values())
    n=[]
    for key, el in scores.items():
        if el==bestscore:
            n=key
    return [ lats[n],longs[n]]
            

In [573]:
def find1(point,category):
     return db.geoprog.find( {"$and":
                              [{ "office" :
                                 { "$near" :
                                   { "$geometry" : point,
                                         "$maxDistance" : 100000} } },
                               {"category_code":f"{category}"}]} )

def nearestpleces(point):
    places=[]
    for cat in categories:
        try:
            el=find1(point,cat)[0]      
            places.append(
                {"latitude" :  el["office"]["coordinates"][1],
                 "longitude" :  el["office"]["coordinates"][0],
                 "name":  el["name"],
                 "category_code": el["category_code"]})           
        except:
            try:
                el=find1(point,cat)[1]      
                places.append(
                    {"latitude" :  el["office"]["coordinates"][1],
                     "longitude" :  el["office"]["coordinates"][0],
                     "name":  el["name"],
                     "category_code": el["category_code"]})
            except:
                continue
    return places

In [504]:
def create_map(lat,long,dbTot):
    m = folium.Map(location=[lat, long],zoom_start=12)

    tooltip = 'Click me!'
    folium.Marker([lat, long], popup='<i>My Company!</i>', tooltip=tooltip).add_to(m)
    
    def fcolors(x):
        colors={"airport":"green","Starbucks":"red","VeganRest":"black","design":"white","company":"darkblue"}
        return colors[x]
    dbTot["colors"]=dbTot["category_code"].apply(fcolors)

    #creation of the map

    marker_cluster = MarkerCluster().add_to(m)

    locations = dbTot[['latitude', 'longitude','colors','category_code']]
    locationlist = locations.values.tolist()

    for pt in range(0, len(locationlist)):
        try:
            folium.Marker(location = locationlist[pt][0:2], popup= locationlist[pt][3], icon=folium.Icon(color=locationlist[pt][2])).add_to(marker_cluster)
        except:
            continue

    display(m)